In [7]:
from typing import Union
from collections import deque
import os
import sys

# For simulation
import random

In [8]:
sys.path.insert(0, os.path.abspath("python_modules"))
from Server import Simulation, ISecurity, OrderSide, LimitOrder, CancelOrder

In [9]:
class MySecurity(ISecurity):
    def __init__(self, name):
        super().__init__()
        self.name = name

    def is_tradeable(self):
        return True

    def before_step(self, simulation):
        print(f"[{self.name}] before_step @ t={simulation.get_t():.2f}")

    def after_step(self, simulation):
        print(f"[{self.name}] after_step @ t={simulation.get_t():.2f}")

    def on_simulation_start(self, simulation):
        print(f"[{self.name}] Simulation starting.")

    def on_simulation_end(self, simulation):
        print(f"[{self.name}] Simulation ended.")

    def on_trade_executed(self, simulation, buyer_id, seller_id, price, volume):
        print(f"[{self.name}] Trade: buyer={buyer_id}, seller={seller_id}, price={price}, volume={volume}")


In [10]:
# Create Python-side securities
security_cad = MySecurity("CAD")
security_stock = MySecurity("STOCK")

# Construct ticker map for the simulation
ticker_to_security = {
    "CAD": security_cad,
    "STOCK": security_stock
}

# Instantiate simulation
sim = Simulation(ticker_to_security, N=1_0, T=1.0)


In [11]:
# Register users
user1 = sim.add_user("alice")
user2 = sim.add_user("bob")

# Create command queues (one per security)
command_queues = {
    sim.get_security_id("CAD"): deque(),
    sim.get_security_id("STOCK"): deque()
}

# Add a bid from alice and ask from bob
command_queues[sim.get_security_id("STOCK")].append(LimitOrder(
    user_id=user1,
    order_id=1,
    side=OrderSide.BID,
    price=100.0,
    volume=10.0
))

command_queues[sim.get_security_id("STOCK")].append(LimitOrder(
    user_id=user2,
    order_id=2,
    side=OrderSide.ASK,
    price=95.0,
    volume=5.0
))


In [12]:
# Perform 4 steps: start, middle, end
while True:
    result = sim.do_simulation_step(command_queues)

    print(f"\n📈 Step {result.current_step} Summary:")
    print("Transactions:")
    for ticker, txs in result.transactions.items():
        for tx in txs:
            print(f"  {ticker}: {tx.buyer_id} bought {tx.volume} from {tx.seller_id} at ${tx.price}")

    if not result.has_next_step:
        break


[CAD] Simulation starting.
[STOCK] Simulation starting.
[CAD] before_step @ t=0.00
[STOCK] before_step @ t=0.00
[STOCK] Trade: buyer=0, seller=1, price=100.0, volume=5.0
[CAD] after_step @ t=0.00
[STOCK] after_step @ t=0.00

📈 Step 0 Summary:
Transactions:
  STOCK: 0 bought 5.0 from 1 at $100.0
[CAD] before_step @ t=0.10
[STOCK] before_step @ t=0.10
[STOCK] Trade: buyer=0, seller=1, price=100.0, volume=5.0
[CAD] after_step @ t=0.10
[STOCK] after_step @ t=0.10

📈 Step 1 Summary:
Transactions:
  STOCK: 0 bought 5.0 from 1 at $100.0
[CAD] before_step @ t=0.20
[STOCK] before_step @ t=0.20
[STOCK] Trade: buyer=0, seller=1, price=100.0, volume=5.0
[CAD] after_step @ t=0.20
[STOCK] after_step @ t=0.20

📈 Step 2 Summary:
Transactions:
  STOCK: 0 bought 5.0 from 1 at $100.0
[CAD] before_step @ t=0.30
[STOCK] before_step @ t=0.30
[STOCK] Trade: buyer=0, seller=1, price=100.0, volume=5.0
[CAD] after_step @ t=0.30
[STOCK] after_step @ t=0.30

📈 Step 3 Summary:
Transactions:
  STOCK: 0 bought 5.0 f